In [69]:
import pixiedust
from __future__ import print_function
import keras
from keras import backend as K
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#text = ';'.join([s + '.' + s[::-1] for s in [''.join(random.choices(['A','B','C','D','E','F','G','H','I'], k=4)) for i in range(0, 100)]])
#text_test = 'ABCD.'

# text = "ABABABABABABABABABABABABABABABABABABABABABABAB"
# text_test = "BABAB"

#text = "ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB.ABB"
#text_test = "BB.ABB.ABB.A"

text = "1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA"
text_test = "ABABA1AABBAAB"

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
one_hot_chars = np.eye(len(chars))

N_train = 1
max_len = len(text)
input_len = len(chars)
lstm_t = 1 * input_len

# print('Vectorization...')
# X_train = np.zeros((N_train, max_len, input_len), dtype=np.bool)
# # Y_train = np.zeros((N_train, input_len), dtype=np.bool)
# for t, char in enumerate(text):
#     X_train[0, t, char_indices[char]] = 1
# # Y_train[0, char_indices[text[t+1]]] = 1


model = Sequential()
model.add(LSTM(lstm_t, batch_input_shape=(1, 1, input_len), return_sequences=False, stateful=True))
model.add(Dense(input_len, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(text)
print()

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch):
    input_text = []
    output_text = []

    for i in range(0, len(text_test) - 1):
        input_char = text_test[i]
        one_hot_input = one_hot_chars[char_indices[input_char]]
        preds = model.predict(np.array([[one_hot_input]]), verbose=0)[0]

    char = text_test[-1]
    for i in range(0, 20):
        input_char = char
        one_hot_input = one_hot_chars[char_indices[input_char]]
        input_text.append(input_char)

        preds = model.predict(np.array([[one_hot_input]]), verbose=0)[0]
        print(preds)

        char = indices_char[sample(preds, 0.01)]
        output_text.append(char)

    #print('input  = {}'.format(''.join(input_text)))
    #print('output = {}'.format(''.join(output_text)))
    print('{} => {}'.format(text_test, ''.join(output_text)))

        
print('Train...')
for epoch in range(500):
    print('Epoch {}'.format(epoch))
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(0, len(text) - 1):
        input_char = text[i]
        output_char = text[i+1]
        #print('{} -> {}'.format(input_char, output_char))
        one_hot_input = one_hot_chars[char_indices[input_char]]
        one_hot_output = one_hot_chars[char_indices[output_char]]

        tr_loss, tr_acc = model.train_on_batch(np.array([[one_hot_input]]),
                                               np.array([one_hot_output]))
        mean_tr_acc.append(tr_acc)
        mean_tr_loss.append(tr_loss)
    model.reset_states()

    on_epoch_end(epoch)
    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')



total chars: 4
1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA1AABBAABB2BABABABA

Train...
Epoch 0
[0.42280588 0.39942846 0.5162581  0.5425595 ]
[0.42820886 0.4082968  0.53545856 0.5490054 ]
[0.43102825 0.41355804 0.54747415 0.55258894]
[0.43242717 0.4168458  0.555225   0.5544641 ]
[0.425959   0.4000515  0.52349854 0.5477318 ]
[0.4257907  0.40410972 0.54003847 0.5529325 ]
[0.42883432 0.41009712 0.55042315 0.5552099 ]
[0.43047953 0.41397747 0.55720615 0.5562902 ]
[0.43143383 0.41672286 0.56177336 0.55665094]
[0.4320142  0.41878125 0.5649444  0.55660814]
[0.4275025  0.40290517 0.53307176 0.55105364]
[0.42551526 0.40493217 0.54560673 0.55440897]
[0.42838275 0.41063428 0.55417424 0.5558825 ]
[0.42359778 0.39577606 0.52295774 0.5492631 ]
[0.42365983 0.40072373 0.53982234 0.554063  ]
[0.42716038 0.4074757  0.5503247  0.55611455]
[0.42914093 0.4119059  0.5571759  0.5570319 ]
[0.42442244 0.39683723 0.5255548  0.550424  ]
[0.4238982  0.40105304 0.54124

[0.08673203 0.08298995 0.4147549  0.4562708 ]
[0.09264392 0.09993163 0.47003993 0.456844  ]
[0.08527734 0.08095062 0.41177702 0.4565502 ]
[0.09120397 0.09771588 0.46724275 0.45706394]
[0.08412819 0.07935224 0.40942553 0.45679006]
[0.09005972 0.09596194 0.4649972  0.45725304]
ABABA1AABBAAB => ABABBABABABABABABABA
accuracy training = 0.75
loss training = 0.439412921667099
___________________________________
Epoch 8
[0.09941653 0.11802431 0.49991608 0.44777223]
[0.09121526 0.09426983 0.4388718  0.4509119 ]
[0.09824883 0.11734154 0.50187886 0.44781372]
[0.09067785 0.09402269 0.44019607 0.45112568]
[0.09771203 0.11692058 0.5025577  0.44798526]
[0.09026922 0.09365388 0.44051465 0.45133042]
[0.09724046 0.11636438 0.5025504  0.44814196]
[0.08985832 0.09315258 0.4402741  0.45150408]
[0.09678167 0.11571771 0.50218815 0.44828174]
[0.08944161 0.09258055 0.43974283 0.45165464]
[0.09633291 0.11503129 0.50164586 0.44840786]
[0.08902869 0.09198284 0.4390698  0.45178843]
[0.0958988  0.11433945 0.501019

[0.0501008  0.05567765 0.4582932  0.4509814 ]
[0.04465743 0.0363815  0.36120924 0.4883212 ]
[0.05064261 0.05703592 0.46334708 0.4498882 ]
[0.04518256 0.03717414 0.36412984 0.48719808]
[0.05126785 0.05852711 0.46800512 0.4484121 ]
[0.04576358 0.03802578 0.36696142 0.48599315]
[0.05191018 0.06005938 0.4725661  0.4469005 ]
[0.04637059 0.03892022 0.36983898 0.4847437 ]
[0.05256766 0.061641   0.47712085 0.44536403]
[0.04700381 0.03986328 0.37281373 0.48345158]
[0.05324268 0.06328234 0.48171505 0.44380012]
[0.04766683 0.04086328 0.37591946 0.48211178]
[0.05393851 0.06499422 0.48637846 0.44220325]
[0.04836445 0.04193    0.37918493 0.480717  ]
[0.05465877 0.06678786 0.49113423 0.44056696]
[0.04910223 0.04307464 0.3826386  0.4792588 ]
[0.05540707 0.06867482 0.49600205 0.4388842 ]
[0.04988642 0.04431002 0.38631046 0.4777276 ]
[0.05618724 0.07066753 0.50099945 0.43714827]
[0.05072407 0.04565099 0.39023295 0.4761128 ]
ABABA1AABBAAB => ABABABABABABABABABAB
accuracy training = 0.7757009267807007
los

[0.04742679 0.03965937 0.37011403 0.50146735]
[0.05325853 0.07262765 0.5261109  0.41335157]
[0.04748751 0.03976419 0.37050354 0.50121135]
[0.05331447 0.07280542 0.52656996 0.41308758]
[0.04755113 0.03987416 0.3709111  0.50094366]
[0.05337291 0.0729913  0.5270483  0.4128121 ]
ABABA1AABBAAB => BABABABABABABABABABA
accuracy training = 0.8014018535614014
loss training = 0.41912543773651123
___________________________________
Epoch 25
[0.04890507 0.05992456 0.4948314  0.4282008 ]
[0.04412126 0.03388958 0.3515973  0.51520574]
[0.05129937 0.06740005 0.5180974  0.41523445]
[0.04656613 0.03790316 0.36737195 0.5038224 ]
[0.05386813 0.07579304 0.5406019  0.40212193]
[0.0495738  0.04326789 0.38730627 0.4903065 ]
[0.05652417 0.08492739 0.561906   0.3892525 ]
[0.05307709 0.05009219 0.41064268 0.47526625]
[0.059323   0.09466583 0.5811457  0.37710002]
[0.05705427 0.05857097 0.43681902 0.45913818]
[0.06293579 0.10563091 0.59686726 0.36620006]
[0.06150325 0.06881808 0.46458468 0.44257748]
[0.04430173 0.

[0.03319069 0.02214812 0.31000686 0.56018394]
[0.04384252 0.05013224 0.47256276 0.43408924]
[0.04053096 0.02517303 0.29853472 0.5782163 ]
[0.04328349 0.04823871 0.46362883 0.43989953]
[0.04001978 0.0244372  0.29465243 0.5823819 ]
[0.04264177 0.04621401 0.4547396  0.4465854 ]
[0.03948139 0.02370456 0.29088613 0.5865542 ]
[0.0419875  0.04419962 0.4455201  0.45354134]
[0.05651843 0.10114679 0.61390895 0.32583198]
[0.05587569 0.05533607 0.42671874 0.46369135]
[0.05651672 0.09889972 0.6050822  0.3300805 ]
[0.0553532  0.05396065 0.4221662  0.46771434]
[0.05608165 0.09735327 0.60273993 0.33216438]
[0.05481621 0.05264955 0.4175481  0.47135115]
[0.05566292 0.0958486  0.60037434 0.33424982]
[0.05429458 0.05139377 0.41304114 0.47492048]
[0.05524835 0.09435993 0.5979686  0.33635238]
[0.05378443 0.0501852  0.408633   0.47843546]
[0.05483635 0.09288286 0.59551764 0.33847764]
[0.05328394 0.04901837 0.40430987 0.4819065 ]
ABABA1AABBAAB => BABABABBABABABABABAB
accuracy training = 0.7897196412086487
los

...

[0.03327677 0.01320807 0.21612109 0.69001937]
[0.03389403 0.03821277 0.44495365 0.4193777 ]
[0.05811285 0.01473035 0.17199998 0.7726693 ]
[0.03368066 0.03667558 0.44287366 0.4289599 ]
[0.0579395  0.01464582 0.17211024 0.7732434 ]
[0.03365849 0.03673321 0.44356465 0.4283879 ]
[0.05793652 0.01467681 0.17240791 0.7728389 ]
[0.03366083 0.03684104 0.44428998 0.42762107]
[0.03604622 0.09182122 0.64286256 0.2228901 ]
[0.06058941 0.04691968 0.39161727 0.50300145]
[0.03579333 0.0900589  0.6433551  0.22566342]
[0.0604844  0.04811458 0.39924607 0.49565792]
[0.03585048 0.09138843 0.64610606 0.2229751 ]
[0.06054302 0.04964888 0.40713513 0.48723295]
[0.03591448 0.09278671 0.64884347 0.2202299 ]
[0.0606051  0.05123926 0.41509932 0.47877613]
[0.0359788  0.09418651 0.65151477 0.21754678]
[0.06066711 0.05287686 0.42311096 0.47033566]
[0.03604312 0.09558319 0.65411526 0.21493202]
[0.06072882 0.05455821 0.43114433 0.46193895]
ABABA1AABBAAB => BABABABBABABABABABAB
accuracy training = 0.8831775784492493
los

[0.0624536  0.02589494 0.2532245  0.669123  ]
[0.03345755 0.06316207 0.56512046 0.3025594 ]
[0.06240732 0.0248993  0.24591604 0.67819256]
[0.03342231 0.06150838 0.5592295  0.3085712 ]
[0.06235904 0.02392312 0.23863944 0.6872932 ]
[0.03338639 0.05981887 0.5529906  0.31494972]
[0.06230868 0.02296802 0.23141228 0.69640326]
ABABA1AABBAAB => BBABABABABABABABABAB
accuracy training = 0.8317757248878479
loss training = 0.3709753453731537
___________________________________
Epoch 96
[0.0317087  0.01614913 0.2695132  0.6145483 ]
[0.03277406 0.05477979 0.5418793  0.31102976]
[0.06073868 0.01945135 0.2178403  0.7043172 ]
[0.0323834  0.05487196 0.5538328  0.30768457]
[0.06059287 0.02080227 0.23043315 0.6889161 ]
[0.03247224 0.05857347 0.56885463 0.29334834]
[0.06067207 0.02257817 0.24468721 0.67050576]
[0.03257853 0.0624972  0.5832181  0.279572  ]
[0.06075454 0.02458828 0.26028064 0.65071905]
[0.03269172 0.06654943 0.5967945  0.2666193 ]
[0.06083481 0.02685125 0.2772152  0.6296577 ]
[0.03281104 0.0

[0.03163835 0.07992635 0.6147537  0.25159615]
[0.06413572 0.03894786 0.34602517 0.56489027]
ABABA1AABBAAB => BBABABABABABABABABAB
accuracy training = 0.7873831987380981
loss training = 0.3987748324871063
___________________________________
Epoch 104
[0.0311306  0.01079663 0.19039303 0.72430605]
[0.03041448 0.03358714 0.41588515 0.4454652 ]
[0.03154643 0.08528171 0.6211276  0.23674251]
[0.06352101 0.03456489 0.31812927 0.59076303]
[0.03097274 0.07658541 0.6118185  0.25406113]
[0.06331616 0.03417904 0.3189022  0.5924663 ]
[0.03096385 0.0766925  0.6124556  0.25369582]
[0.06330846 0.03450158 0.3210958  0.59004503]
[0.03097469 0.07709532 0.61344635 0.25269753]
[0.06330778 0.03485584 0.3233714  0.5874484 ]
[0.03098639 0.07751312 0.6144527  0.25167415]
[0.06330739 0.03521828 0.3256821  0.58481437]
[0.03099837 0.07793611 0.61546266 0.25064725]
[0.06330702 0.03558817 0.32802594 0.5821482 ]
[0.03101047 0.0783637  0.6164752  0.24961792]
[0.06330662 0.03596545 0.33040237 0.5794509 ]
[0.03102277 0.

[0.18145429 0.0544836  0.37831914 0.8042172 ]
ABABA1AABBAAB => BBABABABABABABABABAB
accuracy training = 0.8738317489624023
loss training = 0.34251174330711365
___________________________________
Epoch 113
[0.03033461 0.01512459 0.26006067 0.64072347]
[0.03656934 0.06852212 0.5709184  0.3338924 ]
[0.07711731 0.01966922 0.21282694 0.75595677]
[0.04593464 0.08411027 0.6262188  0.35027143]
[0.09611732 0.02756755 0.26521653 0.75075865]
[0.06306702 0.12284685 0.6953115  0.35224438]
[0.12822942 0.04155398 0.3350991  0.75159395]
[0.08461493 0.16870071 0.7470741  0.3607065 ]
[0.16236757 0.05720382 0.3956334  0.7570591 ]
[0.10169017 0.19185178 0.7615501  0.38476658]
[0.18343134 0.06376522 0.41648203 0.77115846]
[0.11043686 0.19476973 0.75967604 0.40818566]
[0.19349863 0.06340552 0.41339564 0.7867495 ]
[0.1140381  0.18835019 0.7513759  0.428953  ]
[0.19713025 0.05977312 0.39906174 0.8015539 ]
[0.11478285 0.17759533 0.73933226 0.44831622]
[0.19679168 0.054825   0.37953922 0.8148838 ]
[0.11374379 0

KeyboardInterrupt: 